#### Summary

* Bidirectional LSTM 15 hidden units.
* Batch Normalitzation.
* Data padded at the end of the sequences.
* Cross validation 10 folds.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_json('train.json')
df_test = pd.read_json('test.json')
df_train.head()

,audio_embedding,end_time_seconds_youtube_clip,is_turkey,start_time_seconds_youtube_clip,vid_id
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",70,0,60,kDCk3hLIVXo
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",40,1,30,DPcGzqHoo7Y
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",240,1,230,7yM63MTHh5k
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",520,1,510,luG3RmUAxxM
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",10,0,0,PIm3cjxTpOk


Some parameters

In [3]:
'''
NxTxD
T = 10 #rows, seq. length
D = 128 #columns, input dim.
N = 1195 (train) #samples
k = 2 {0,1} #classes
M = ?, LSTM latent dim.
'''
T = 10
D = 128
N = None
k = 1
M = 15

In [4]:
from keras.layers import LSTM, Bidirectional, Input, Dense, Flatten, BatchNormalization
from keras.models import Model

Using TensorFlow backend.


In [5]:
main_input = Input(shape=(T,D,))
lstm = Bidirectional(LSTM(M, return_sequences=True))(main_input)
flat = Flatten()(lstm)
bn = BatchNormalization()(flat)
main_output = Dense(1, activation='sigmoid')(bn)

In [6]:
model = Model(inputs=[main_input], outputs=[main_output])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 300)           334800    
_________________________________________________________________
flatten_1 (Flatten)          (None, 3000)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3000)              12000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3001      
Total params: 349,801
Trainable params: 343,801
Non-trainable params: 6,000
_________________________________________________________________


In [8]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
from keras.preprocessing.sequence import pad_sequences

X_train_list = []
y_train_list = []
for i in range(len(df_train)):
    X_train = np.array(df_train.loc[i,'audio_embedding'])
    #If the sample doesn't have 10 seconds, we instert zeros at the end of the sequence
    X_train = pad_sequences(X_train.T, maxlen=10, padding='post').T
    X_train = X_train.reshape(10,128)
    X_train_list.append(X_train)
    
    y_train = np.array(df_train.loc[i,'is_turkey'])
    y_train = y_train.reshape(1,)
    y_train_list.append(y_train)

Cross validation with 10 folds:

In [10]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)

X_train_list = np.array(X_train_list)
y_train_list = np.array(y_train_list)

for train_index, test_index in kf.split(X_train_list):
    Xtrain = X_train_list[train_index]
    Xtest = X_train_list[test_index]
    ytrain = y_train_list[train_index]
    ytest = y_train_list[test_index]
    
    model.fit(np.array(Xtrain), np.array(ytrain), epochs=10, batch_size=128, validation_data=(Xtest, ytest))


Train on 1075 samples, validate on 120 samples
Epoch 1/20
1075/1075 [==============================] - 6s 5ms/step - loss: 0.3695 - acc: 0.8288 - val_loss: 0.1456 - val_acc: 0.9667
Epoch 2/20
1075/1075 [==============================] - 1s 584us/step - loss: 0.1800 - acc: 0.9340 - val_loss: 0.0921 - val_acc: 0.9667
Epoch 3/20
1075/1075 [==============================] - 1s 590us/step - loss: 0.1654 - acc: 0.9358 - val_loss: 0.0956 - val_acc: 0.9833
Epoch 4/20
1075/1075 [==============================] - 1s 578us/step - loss: 0.1395 - acc: 0.9526 - val_loss: 0.1027 - val_acc: 0.9500
Epoch 5/20
1075/1075 [==============================] - 1s 573us/step - loss: 0.1397 - acc: 0.9498 - val_loss: 0.1470 - val_acc: 0.9417
Epoch 6/20
1075/1075 [==============================] - 1s 584us/step - loss: 0.1251 - acc: 0.9516 - val_loss: 0.1130 - val_acc: 0.9583
Epoch 7/20
1075/1075 [==============================] - 1s 596us/step - loss: 0.1195 - acc: 0.9516 - val_loss: 0.3584 - val_acc: 0.8667
Epo

Epoch 20/20
1075/1075 [==============================] - 1s 571us/step - loss: 0.0571 - acc: 0.9851 - val_loss: 0.2282 - val_acc: 0.9333
Train on 1075 samples, validate on 120 samples
Epoch 1/20
1075/1075 [==============================] - 1s 578us/step - loss: 0.0576 - acc: 0.9777 - val_loss: 0.1294 - val_acc: 0.9250
Epoch 2/20
1075/1075 [==============================] - 1s 566us/step - loss: 0.0882 - acc: 0.9702 - val_loss: 0.0684 - val_acc: 0.9917
Epoch 3/20
1075/1075 [==============================] - 1s 578us/step - loss: 0.0587 - acc: 0.9702 - val_loss: 0.1382 - val_acc: 0.9583
Epoch 4/20
1075/1075 [==============================] - 1s 590us/step - loss: 0.0802 - acc: 0.9674 - val_loss: 0.1136 - val_acc: 0.9500
Epoch 5/20
1075/1075 [==============================] - 1s 591us/step - loss: 0.0646 - acc: 0.9823 - val_loss: 0.3204 - val_acc: 0.8667
Epoch 6/20
1075/1075 [==============================] - 1s 574us/step - loss: 0.0856 - acc: 0.9758 - val_loss: 0.1721 - val_acc: 0.9167


Epoch 19/20
1076/1076 [==============================] - 1s 574us/step - loss: 0.0383 - acc: 0.9898 - val_loss: 0.2766 - val_acc: 0.9076
Epoch 20/20
1076/1076 [==============================] - 1s 566us/step - loss: 0.0602 - acc: 0.9805 - val_loss: 0.0778 - val_acc: 0.9496
Train on 1076 samples, validate on 119 samples
Epoch 1/20
1076/1076 [==============================] - 1s 631us/step - loss: 0.0492 - acc: 0.9833 - val_loss: 0.0212 - val_acc: 0.9916
Epoch 2/20
1076/1076 [==============================] - 1s 590us/step - loss: 0.0475 - acc: 0.9805 - val_loss: 0.0479 - val_acc: 0.9748
Epoch 3/20
1076/1076 [==============================] - 1s 598us/step - loss: 0.0489 - acc: 0.9814 - val_loss: 0.0561 - val_acc: 0.9748
Epoch 4/20
1076/1076 [==============================] - 1s 568us/step - loss: 0.0658 - acc: 0.9749 - val_loss: 0.1223 - val_acc: 0.9832
Epoch 5/20
1076/1076 [==============================] - 1s 579us/step - loss: 0.0705 - acc: 0.9758 - val_loss: 0.0447 - val_acc: 0.9832

Epoch 18/20
1076/1076 [==============================] - 1s 612us/step - loss: 0.0614 - acc: 0.9768 - val_loss: 0.0271 - val_acc: 0.9832
Epoch 19/20
1076/1076 [==============================] - 1s 613us/step - loss: 0.0368 - acc: 0.9851 - val_loss: 0.0495 - val_acc: 0.9832
Epoch 20/20
1076/1076 [==============================] - 1s 598us/step - loss: 0.0393 - acc: 0.9861 - val_loss: 0.0986 - val_acc: 0.9832
Train on 1076 samples, validate on 119 samples
Epoch 1/20
1076/1076 [==============================] - 1s 582us/step - loss: 0.0502 - acc: 0.9805 - val_loss: 0.0332 - val_acc: 0.9832
Epoch 2/20
1076/1076 [==============================] - 1s 603us/step - loss: 0.0446 - acc: 0.9842 - val_loss: 0.0440 - val_acc: 0.9916
Epoch 3/20
1076/1076 [==============================] - 1s 617us/step - loss: 0.0373 - acc: 0.9888 - val_loss: 0.0579 - val_acc: 0.9580
Epoch 4/20
1076/1076 [==============================] - 1s 599us/step - loss: 0.0345 - acc: 0.9907 - val_loss: 0.0730 - val_acc: 0.974

#### Predictions part

In [11]:
df_test.head()

,audio_embedding,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip,vid_id
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",10,0,pyKh38FXD3E
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",40,30,THhP1idrWXA
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",40,30,jsw3T6GY2Nw
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",24,14,nFkXTMHcjMU
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",40,30,Au8g9kAlrLQ


Padding the new data like we did during training.

In [12]:
pred_list = []
for i in range(len(df_test)):
    pred = np.array(df_test.loc[i,'audio_embedding'])
    pred = pad_sequences(pred.T, maxlen=10, padding='post').T
    pred = pred.reshape(10,128)
    pred_list.append(pred)

In [13]:
preds = model.predict(np.array(pred_list))

In [14]:
df_test['is_turkey'] = preds

Save the results

In [15]:
df_test[['vid_id', 'is_turkey']].to_csv('my_dinner.csv', index=False)